# 필수 모듈 가져오기

In [96]:
from numpy import int16
import requests
import json
from playsound import playsound
from scipy.io.wavfile import write
import sounddevice
import multiprocessing
from datetime import datetime

# 음성 녹음

In [97]:
fs=16000
second=5
print("recording....")
record_voice=sounddevice.rec(int(second * fs), samplerate=fs, channels=1, dtype=int16)
sounddevice.wait()
write('temp.wav', fs, record_voice)

recording....


# STT 서비스 요청

In [98]:
url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"
key = "58df5a2afd5c776e6d4e95f5cabf709e"
headers = {
    "Content-Type": "application/octet-stream",
    "Authorization": "KakaoAK " + key,
}
with open('temp.wav', 'rb') as fp:
    audio = fp.read()
res = requests.post(url, headers=headers, data=audio)

# 요청 결과 출력

In [99]:
json_string = res.text[res.text.index('{"type":"finalResult"'):res.text.rindex('}')+1]
result = json.loads(json_string)['value']
print(result)

지금 몇 시야


# 키워드 비교

In [100]:
keyword = ''
result = result.replace(" ","")
if "몇시" in result:
    keyword = 'CURRENT_TIME'
else:
    keyword = 'UNKNOWN'

# 키워드 출력

In [101]:
print(keyword)

CURRENT_TIME


# 키워드가 CURRENT_TIME이라면 현재 시각을 data로 설정

In [102]:
data = ''
if keyword == 'CURRENT_TIME':
    now = datetime.today().strftime('%H:%M')
    data = f'<speak>현재 시각은<break time="150ms"/>\
              <say-as interpret-as="time" format="hm24"> {now} </say-as> 입니다</speak>'
data = data.encode('utf-8').decode('latin1')

# TTS 서비스 요청

In [103]:
url = "https://kakaoi-newtone-openapi.kakao.com/v1/synthesize"
key = "58df5a2afd5c776e6d4e95f5cabf709e"
headers = {
    "Content-Type": "application/xml",
    "Authorization": "KakaoAK " + key,
}
res = requests.post(url, headers=headers, data=data)
if (res.status_code != 200):
    print(f'TTS 요청이 실패하였습니다.(code={res.status_code})')
else:
    with open('result.mp3', "wb") as f:
        f.write(res.content)

# TTS 서비스 결과 출력

In [104]:
p = multiprocessing.Process(target=playsound, args=('result.mp3',))
p.start()
input("press ENTER to stop playback")
p.terminate()

press ENTER to stop playback
